# 1. 데이터 train_data와 test_test로 분류하기

 - datset에는 총 16000장의 병충해 이미지 데이터가 있다.
 - label.json에는 총 12800장에 대한 라벨링이 되어 있음 = train_data로 이용
 - 따라서 datset에서 label.json에 없는 이미지(라벨링이 되지 않은 이미지)를 따로 빼서 test_data에 저장해야한다.
 - 이후 ImageFolder를 이용하기 위해 train_data를 라벨별로 폴더를 생성하여 나눠준다.

In [29]:
import os
import json

path_dir = './datset'

file_list = os.listdir(path_dir) # 사이즈 16000개
test_list = os.listdir(path_dir)

file = open('./label.json')
jsonString = json.load(file)
train_list = []
for i in jsonString['train']:
    train_list.append(i[0])
# train_list 총 12800개

for i in train_list:
    test_list.remove(i)
# 전체 데이터에서 train데이터삭제 후 남은 데이터 = test데이터(3200개)

In [30]:
import shutil
#src에 있는 파일을 dir로 이동.
src = './datset/'
dir_test = './test_data/'
dir_train = './train_data/'
for filename in test_list:
    shutil.move(src+filename, dir_test+filename)

for filename in train_list:
    shutil.move(src+filename, dir_train+filename)

# 이미지 폴더로 분류 완료

In [31]:
# train_data에서 라벨별로 분류

import os
import json
import shutil
path_dir = './train_data'

file_list = os.listdir(path_dir) # 사이즈 12800개
file = open('./label.json')
jsonString = json.load(file)

for file_name, label in jsonString['train']:
    src = './train_data/' + file_name
    dest = './train_data/' + str(label) +'/'
    if not os.path.exists(dest):
        os.makedirs(dest)
    else:
        shutil.move(src, dest+file_name)

In [32]:
num6=num8=num11=num13=num14=num15=num17=num19=num22=num23=num25=num27=num28=num29=num30=num31=num33 = 0
for file_name, label in jsonString['train']:
    if label == 6:
        num6 += 1
    elif label == 8:
        num8 += 1
    elif label == 11:
        num11 += 1
    elif label == 13:
        num13 += 1
    elif label == 14:
        num14 += 1
    elif label == 15:
        num15 += 1
    elif label == 17:
        num17 += 1
    elif label == 19:
        num19 += 1
    elif label == 22:
        num22 += 1
    elif label == 23:
        num23 += 1
    elif label == 25:
        num25 += 1
    elif label == 27:
        num27 += 1
    elif label == 28:
        num28 += 1  
    elif label == 29:
        num29 += 1    
    elif label == 30:
        num30 += 1
    elif label == 31:
        num31 += 1
    elif label == 33:
        num33 += 1                         
print(num6, num8, num11, num13, num14, num15, num17, num19, num22, num23, num25, num27, num28, num29, num30, num31, num33)
print(num6+num8+num11+num13+num14+num15+num17+num19+num22+num23+num25+num27+num28+num29+num30+num31+num33)

639 1290 638 647 1304 640 650 628 641 628 643 644 626 626 1275 633 648
12800


# 2. train dataset 이미지를 전처리하기

- transpose를 이용해서 전처리하기
- image augmentation을 이용하기
- 딥러닝 모델에 맞게 사이즈 줄이기


In [34]:
import torchvision
import torch
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader


In [35]:
trans = transform.Compose([transforms.Resize((28,28)),
                                transforms.ToTensor(), 
                                transforms.Normalize((0.5,),(0.5,))
                                ])

train_data = datasets.ImageFolder(root = './train_data/', transform = trans)

In [38]:
for num, value in enumerate(train_data):
    data, label = value
    print(num, data, label)

.Image image mode=RGB size=256x256 at 0x2C2DD8479C8> 16
12523 <PIL.Image.Image image mode=RGB size=256x256 at 0x2C2DD847A88> 16
12524 <PIL.Image.Image image mode=RGB size=256x256 at 0x2C2DD847B48> 16
12525 <PIL.Image.Image image mode=RGB size=256x256 at 0x2C2DD847C08> 16
12526 <PIL.Image.Image image mode=RGB size=256x256 at 0x2C2DD847CC8> 16
12527 <PIL.Image.Image image mode=RGB size=256x256 at 0x2C2DD847D88> 16
12528 <PIL.Image.Image image mode=RGB size=256x256 at 0x2C2DD847E48> 16
12529 <PIL.Image.Image image mode=RGB size=256x256 at 0x2C2DD847F08> 16
12530 <PIL.Image.Image image mode=RGB size=256x256 at 0x2C2DD847FC8> 16
12531 <PIL.Image.Image image mode=RGB size=256x256 at 0x2C2DD8510C8> 16
12532 <PIL.Image.Image image mode=RGB size=256x256 at 0x2C2DD851188> 16
12533 <PIL.Image.Image image mode=RGB size=256x256 at 0x2C2DD851248> 16
12534 <PIL.Image.Image image mode=RGB size=256x256 at 0x2C2DD851308> 16
12535 <PIL.Image.Image image mode=RGB size=256x256 at 0x2C2DD8513C8> 16
12536 <P